### Preprocessing
The aim of this part is to extract data from the HF, rephraze it and clean it. 

In [6]:
import pandas as pd
import re
from pandas.core.interchange.from_dataframe import primitive_column_to_ndarray
import os

In [5]:
# load the dataset from HF
df_full = pd.read_parquet("hf://datasets/ruggsea/stanford-encyclopedia-of-philosophy_chat_multi_turn/data/train-00000-of-00001.parquet")

# I recommend to work on mock dataset (50 first rows) for writing the code
df = df_full.head(5)
df.head()

ArrowKeyError: No type extension with name arrow.py_extension_type found

In [11]:
# set univrsal working directory 
root_name = "zipf-law-for-information-theory"
current_dir = os.getcwd()

if os.path.basename(current_dir) != root_name:
    os.chdir(os.path.dirname(current_dir))

print("working path:", os.getcwd())

working path: C:\zipf-law-for-information-theory


In [13]:
def extract_content(conversation: dict):
    """
    Unwraps original dictionary with student's and professor utterances.
    :param conversation: dict with utterances
    :return: list of professor utterances and student utterances
    """
    professor = []
    user = []
    for turn in conversation:
        # print(turn)
        # print('-'*20)
        if turn['role'] == 'user':
            user.append(turn['content'])
        else:
            professor.append(turn['content'])
    return professor, user

def prepare_prompt(prompt: str):
    """
    Cleans the redundant content from each prompt. Desgined to use with lambda.
    :param prompt: prompt to clean
    :return: cleaned prompt
    """
    return re.findall(r'\"([\s\S]+?)\"', prompt)


In [100]:
# Split the conversation into various rows in df
# select utterances of professor
df['professor'] = df['conversation'].apply(lambda x: extract_content(x)[0])

# select utterances of student
df['student'] = df['conversation'].apply(lambda x: extract_content(x)[1])

# clean redundant content from prompt
df['prompt'] = df['prompt'].apply(lambda x: prepare_prompt(x))

C:\Users\quiz0\AppData\Local\Temp\ipykernel_27332\959540336.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['professor'] = df['conversation'].apply(lambda x: extract_content(x)[0])
C:\Users\quiz0\AppData\Local\Temp\ipykernel_27332\959540336.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['student'] = df['conversation'].apply(lambda x: extract_content(x)[1])
C:\Users\quiz0\AppData\Local\Temp\ipykernel_27332\959540336.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [101]:
df = df.drop('conversation', axis=1)

In [109]:
df = df.explode(["professor", "student"]) # powerful line! creates a pair of student-professor utterance for each row
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 0 to 4
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   prompt     21 non-null     object
 1   professor  21 non-null     object
 2   student    21 non-null     object
dtypes: object(3)
memory usage: 672.0+ bytes


In [112]:
# Now data is clean!
df.head()
df.to_csv('dfs/preprocessed-df.csv', index=False) # save it for future work

### Tokenization
Add tokenized version of the text for each column.

In [ ]:
from source import tokenizator as tk

In [17]:
# alternative
df = pd.read_csv('dfs/preprocessed-df.csv', delimiter = ',')

In [18]:
df


,prompt,professor,student
0,['You happen to know that Tim and Harry have r...,"That's a great question! You see, our brains a...","Professor, I was thinking about how I conclude..."
1,['You happen to know that Tim and Harry have r...,"Of course, you could be wrong! But here's the ...","But isn't that just an assumption? I mean, I c..."
2,['You happen to know that Tim and Harry have r...,"Ah, great example! This is where the concept o...","That makes sense, I guess. But what about case..."
3,['You happen to know that Tim and Harry have r...,Exactly! Our minds are wired to seek patterns ...,"Yeah, I think so. It's like, our brains are al..."
4,['\n Even if it is true that we routinely rely...,"Well, my inquisitive student, that's a great q...","Professor Phil, do we always choose the simple..."
5,['\n Even if it is true that we routinely rely...,"Ah, an astute observation! Yes, when dealing w...","That's a great point, Professor. But what abou..."
6,['\n Even if it is true that we routinely rely...,"Ah, your colleague is onto something! Yes, sim...",That reminds me of a conversation I had with a...
7,['\n Even if it is true that we routinely rely...,"Ah, excellent question! The critics of abducti...","I see what you mean, Professor. It's like we n..."
8,['\n The Buddha’s discourses collected in the ...,"Ah, you're onto something. Think of a movie. A...","Professor Phil, I was reading this book about ..."
9,['\n The Buddha’s discourses collected in the ...,That's a great point. The Abhidharma philosoph...,But if everything is made up of these fleeting...
